### Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth  # Do this in local & cloud setups
else:
    import torch; v = re.match(r'[\d]{1,}\.[\d]{1,}', str(torch.__version__)).group(0)
    xformers = 'xformers==' + {'2.10':'0.0.34','2.9':'0.0.33.post1','2.8':'0.0.32.post2'}.get(v, "0.0.34")
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth_zoo bitsandbytes accelerate {xformers} peft trl triton unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-4B-Instruct-2507-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-Thinking-2507-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Thinking-2507",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "YOUR_HF_TOKEN", # HF Token for gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2026.2.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Qwen-3` format for conversation style finetunes. We use the [Open Math Reasoning](https://huggingface.co/datasets/unsloth/OpenMathReasoning-mini) dataset which was used to win the [AIMO](https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/leaderboard) (AI Mathematical Olympiad - Progress Prize 2) challenge! We sample 10% of verifiable reasoning traces that used DeepSeek R1, and which got > 95% accuracy. Qwen-3 renders multi turn conversations like below:

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```
We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-thinking",
)

In [5]:
import json
from datasets import Dataset

# 1. Читаем и парсим твой кастомный JSON
data = []
with open('dataset.json', 'r', encoding='utf-8') as f:
    text = f.read()

decoder = json.JSONDecoder()
pos = 0
while pos < len(text):
    # Пропускаем пробелы и переносы строк
    while pos < len(text) and text[pos].isspace():
        pos += 1
    if pos >= len(text):
        break

    # Извлекаем ровно один JSON-объект
    obj, index = decoder.raw_decode(text[pos:])
    data.append(obj)
    pos += index

# 2. Создаем датасет Hugging Face из списка
dataset = Dataset.from_list(data)
print(f"✅ Успешно загружено диалогов: {len(dataset)}")

# 3. Применяем Chat Template (форматируем под Qwen)
def formatting_prompts_func(examples):
   convos = examples["messages"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

# Выводим первый пример, чтобы убедиться, что всё ок
print("\nПример первого диалога в датасете:")
print(dataset[0]['text'][:500] + "...") # выводим первые 500 символов

✅ Успешно загружено диалогов: 33


Map:   0%|          | 0/33 [00:00<?, ? examples/s]


Пример первого диалога в датасете:
<|im_start|>system
Ты эксперт по Luau и Roblox разработке. Отвечаешь точно, с примерами кода, по темам Studio, executor API, и игровых систем.<|im_end|>
<|im_start|>user
How do I detect RemoteSpy scripts? Luau<|im_end|>
<|im_start|>assistant
<think>
The user is asking about detecting RemoteSpy scripts in Luau, which is the scripting language used in Roblox. RemoteSpy is a type of exploit tool that monitors RemoteEvent and RemoteFunction calls in Roblox games.

This appears to be a legitimate sec...


Let's see how the chat template did!

In [6]:
dataset[32]['text']

'<|im_start|>system\nТы эксперт по Luau и Roblox разработке. Отвечаешь точно, с примерами кода, по темам Studio, executor API, и игровых систем.<|im_end|>\n<|im_start|>user\nможно ли декомпилировать любой защищенный luau скрипт на инжектор? типа написать эмулятор типа роблокса луау и оттуда уже в рантайме скрипт вытаскивать? или не так работает<|im_end|>\n<|im_start|>assistant\n<think>\nThe user is asking about whether it\'s possible to decompile any protected Luau script for an injector by writing an emulator-like environment for Roblox Luau and extracting the script at runtime.\n\nLet me think about this technically:\n\n1. **Luau bytecode protection**: Roblox uses Luau, which compiles scripts to bytecode. The bytecode format has changed over time and Roblox applies various protections including encryption of bytecode.\n\n2. **The idea**: The user is asking if you could create a custom Luau VM/emulator that mimics Roblox\'s environment, feed it the protected bytecode, and then extract

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [7]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        max_seq_length = 8192,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/33 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [8]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=6):   0%|          | 0/33 [00:00<?, ? examples/s]

Filter (num_proc=6):   0%|          | 0/33 [00:00<?, ? examples/s]

Unsloth: Removed 1 out of 33 samples from train_dataset where all labels were -100 (no response found after truncation). This prevents NaN loss during training.


Let's verify masking the instruction part is done! Let's print the 100th row again.

In [10]:
tokenizer.decode(trainer.train_dataset[31]["input_ids"])

'<|im_start|>system\nТы эксперт по Luau и Roblox разработке. Отвечаешь точно, с примерами кода, по темам Studio, executor API, и игровых систем.<|im_end|>\n<|im_start|>user\nможно ли декомпилировать любой защищенный luau скрипт на инжектор? типа написать эмулятор типа роблокса луау и оттуда уже в рантайме скрипт вытаскивать? или не так работает<|im_end|>\n<|im_start|>assistant\n<think>\nThe user is asking about whether it\'s possible to decompile any protected Luau script for an injector by writing an emulator-like environment for Roblox Luau and extracting the script at runtime.\n\nLet me think about this technically:\n\n1. **Luau bytecode protection**: Roblox uses Luau, which compiles scripts to bytecode. The bytecode format has changed over time and Roblox applies various protections including encryption of bytecode.\n\n2. **The idea**: The user is asking if you could create a custom Luau VM/emulator that mimics Roblox\'s environment, feed it the protected bytecode, and then extract

Now let's print the masked out example - you should see only the answer is present:

In [12]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[31]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                   <think>\nThe user is asking about whether it\'s possible to decompile any protected Luau script for an injector by writing an emulator-like environment for Roblox Luau and extracting the script at runtime.\n\nLet me think about this technically:\n\n1. **Luau bytecode protection**: Roblox uses Luau, which compiles scripts to bytecode. The bytecode format has changed over time and Roblox applies various protections including encryption of bytecode.\n\n2. **The idea**: The user is asking if you could create a custom Luau VM/emulator that mimics Roblox\'s environment, feed it the protected bytecode, and then extract/decompile the script from there.\n\n\n3. **How Roblox script protection actually works**: Scripts get compiled to bytecode server-side, then encrypted before reaching the client, where they\'re decrypted and executed in the Luau VM with additional obfuscation laye

In [13]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.563 GB.
3.816 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32 | Num Epochs = 2 | Total steps = 8
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.010400
2,0.963300
3,1.019700
4,0.649400
5,0.604400
6,0.979100
7,0.797200
8,0.704200


In [15]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

430.4375 seconds used for training.
7.17 minutes used for training.
Peak reserved memory = 14.393 GB.
Peak reserved memory for training = 10.577 GB.
Peak reserved memory % of max memory = 98.833 %.
Peak reserved memory for training % of max memory = 72.629 %.


In [16]:
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 30.6 MB/s eta 0:00:00


In [17]:
import telebot
import torch

# --- КОНФИГУРАЦИЯ ---
TOKEN = "8677380017:AAFFY7nIjxmZF6szTo4818WTSlHN8c5Rs-U"
ALLOWED_USERS = {1583912819, 6373815735}

SYSTEM_PROMPT = "Ты эксперт по Luau и Roblox разработке. Отвечаешь точно, с примерами кода, по темам Studio, executor API, и игровых систем."

bot = telebot.TeleBot(TOKEN)

# Словарь для хранения контекста каждого пользователя
user_contexts = {}

def get_context(user_id):
    """Возвращает историю диалога пользователя или создает новую."""
    if user_id not in user_contexts:
        user_contexts[user_id] = [{"role": "system", "content": SYSTEM_PROMPT}]
    return user_contexts[user_id]

def is_allowed(message):
    """Проверка прав доступа."""
    return message.from_user.id in ALLOWED_USERS

def send_long_message(chat_id, text):
    """Telegram имеет лимит в 4096 символов. Эта функция разбивает длинные сообщения."""
    max_length = 4000
    for i in range(0, len(text), max_length):
        bot.send_message(chat_id, text[i:i+max_length])

# --- ОБРАБОТЧИКИ КОМАНД ---

@bot.message_handler(commands=['start', 'clear'])
def handle_clear(message):
    if not is_allowed(message):
        bot.reply_to(message, "У вас нет доступа к этому боту. 🚫")
        return

    user_id = message.from_user.id
    # Очищаем контекст, оставляя только системный промпт
    user_contexts[user_id] = [{"role": "system", "content": SYSTEM_PROMPT}]
    bot.reply_to(message, "🧹 Контекст диалога очищен! Я готов к новым вопросам по Luau и Roblox.")

# --- ОБРАБОТЧИК ТЕКСТА (ГЕНЕРАЦИЯ) ---

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    if not is_allowed(message):
        return

    user_id = message.from_user.id
    user_text = message.text

    # Получаем контекст пользователя и добавляем новый вопрос
    context = get_context(user_id)
    context.append({"role": "user", "content": user_text})

    # Отправляем сообщение-заглушку, чтобы юзер знал, что бот думает
    processing_msg = bot.reply_to(message, "⏳ Думаю... (генерация может занять некоторое время)")

    try:
        # 1. Форматируем промпт через шаблон Qwen
        prompt = tokenizer.apply_chat_template(
            context,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=True # Включаем теги <think>
        )

        # 2. Токенизируем
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        input_length = inputs.input_ids.shape[1]

        # 3. Генерируем ответ
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048, # Максимальная длина ответа
            temperature=0.7,
            top_p=0.8,
            top_k=20,
            pad_token_id=tokenizer.eos_token_id
        )

        # 4. Декодируем только сгенерированную часть (обрезаем входной промпт)
        generated_tokens = outputs[0, input_length:]
        response_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

        # 5. Сохраняем ответ бота в историю (контекст)
        context.append({"role": "assistant", "content": response_text})

        # 6. Удаляем сообщение "⏳ Думаю..." и отправляем реальный ответ
        bot.delete_message(message.chat.id, processing_msg.message_id)
        send_long_message(message.chat.id, response_text)

    except Exception as e:
        bot.edit_message_text(f"❌ Произошла ошибка при генерации: {str(e)}",
                              chat_id=message.chat.id,
                              message_id=processing_msg.message_id)
        # Если произошла ошибка, удаляем последний вопрос юзера из контекста,
        # чтобы он не сломал следующие генерации
        context.pop()

# --- ЗАПУСК БОТА ---
print("🤖 Бот запущен и готов к работе! Напиши ему в Telegram.")
bot.infinity_polling(timeout=10, long_polling_timeout=5)

🤖 Бот запущен и готов к работе! Напиши ему в Telegram.


2026-02-24 17:47:08,286 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2026-02-24 17:47:08,290 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for instruct inference are `temperature = 0.7, top_p = 0.8, top_k = 20`

For reasoning chat based inference, `temperature = 0.6, top_p = 0.95, top_k = 20`

In [ ]:
messages = [
    {"role" : "user", "content" : "Solve (x + 2)^2 = 0."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 450, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|im_start|>user
Solve (x + 2)^2 = 0.<|im_end|>
<|im_start|>assistant
<think>
<|im_start|>user
Solve (x + 2)^2 = 0.<|im_end|>
<|im_start|>assistant
<think>
Okay, let's see. I need to solve the equation (x + 2)^2 = 0. Hmm, so first, I remember that when you have something squared equals zero, the only way that can happen is if the base is zero. Because if you square any non-zero number, you get a positive number, right? So if (x + 2)^2 is zero, then x + 2 must be zero. 

So, setting x + 2 equal to zero, I can solve for x by subtracting 2 from both sides. That would give x = -2. But wait, since it's squared, does that mean there's only one solution? Because sometimes quadratic equations have two solutions, but in this case, since it's a perfect square, maybe it's a repeated root. 

Let me check. If I expand (x + 2)^2, that's x^2 + 4x + 4. So the equation becomes x^2 + 4x + 4 = 0. Then, factoring that, it's (x + 2)(x + 2) = 0, which is the same as (x + 2)^2 = 0. So, yeah, the only solutio

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Hugging Face's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("qwen_lora")  # Local saving
tokenizer.save_pretrained("qwen_lora")
# model.push_to_hub("your_name/qwen_lora", token = "YOUR_HF_TOKEN") # Online saving
# tokenizer.push_to_hub("your_name/qwen_lora", token = "YOUR_HF_TOKEN") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "qwen_lora", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens. See [our docs](https://unsloth.ai/docs/basics/inference-and-deployment) for more deployment options.

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("qwen_finetune_16bit", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("HF_USERNAME/qwen_finetune_16bit", tokenizer, save_method = "merged_16bit", token = "YOUR_HF_TOKEN")

# Merge to 4bit
if False:
    model.save_pretrained_merged("qwen_finetune_4bit", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("HF_USERNAME/qwen_finetune_4bit", tokenizer, save_method = "merged_4bit", token = "YOUR_HF_TOKEN")

# Just LoRA adapters
if False:
    model.save_pretrained("qwen_lora")
    tokenizer.save_pretrained("qwen_lora")
if False: # Pushing to HF Hub
    model.push_to_hub("HF_USERNAME/qwen_lora", token = "YOUR_HF_TOKEN")
    tokenizer.push_to_hub("HF_USERNAME/qwen_lora", token = "YOUR_HF_TOKEN")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [docs page](https://unsloth.ai/docs/basics/inference-and-deployment/saving-to-gguf)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("qwen_finetune", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("HF_USERNAME/qwen_finetune", tokenizer, token = "YOUR_HF_TOKEN")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("qwen_finetune", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("HF_USERNAME/qwen_finetune", tokenizer, quantization_method = "f16", token = "YOUR_HF_TOKEN")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("qwen_finetune", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("HF_USERNAME/qwen_finetune", tokenizer, quantization_method = "q4_k_m", token = "YOUR_HF_TOKEN")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "HF_USERNAME/qwen_finetune", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "YOUR_HF_TOKEN", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `qwen_finetune.Q8_0.gguf` file or `qwen_finetune.Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other resources:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
4. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://unsloth.ai/docs/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://unsloth.ai/docs/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).